In [43]:
NAME = "Aung"

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%pip install transformers
%pip install sentence-transformers

     ---------------------------------------- 7.2/7.2 MB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 1.8 MB/s eta 0:00:00
     -------------------------------------- 268.8/268.8 kB 1.8 MB/s eta 0:00:00
     -------------------------------------- 153.2/153.2 kB 1.8 MB/s eta 0:00:00
     -------------------------------------- 266.4/266.4 kB 2.0 MB/s eta 0:00:00
     -------------------------------------- 268.7/268.7 kB 2.1 MB/s eta 0:00:00
     -------------------------------------- 143.0/143.0 kB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
     -------------------------------------- 977.7/977.7 kB 8.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_tra

In [1]:
import pandas as pd
import numpy as np

from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None

import torch
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
from sentence_transformers import SentenceTransformer, util

from sklearn.metrics import accuracy_score

import os
import io
import glob
import zipfile
import tarfile

C:\Users\Insaf\anaconda3\envs\competition01\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
C:\Users\Insaf\anaconda3\envs\competition01\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\Insaf\anaconda3\envs\competition01\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` i

In [47]:
PATH = "/content/drive/MyDrive/SemEval_Data/SplitData/"

In [2]:
mycwd = os.path.dirname(os.getcwd())
# flags
language = "english"
#language = "persian"
#language = "italian"
search_engine = "qwant"
search_engines = ["qwant", "bing", "ecosia"]
#search_engines = ["qwant", "ecosia"]

# folders
folder_images = os.path.join(mycwd, "0-dataset", "test_images")
root_txt =  os.path.join(mycwd, "0-dataset", "test.data.v1.1.gold")
#folder_search = os.path.join(mycwd, "9-Insaf_scrapSearch", "scrape_"+language, search_engine, "query")

if language == "english":
    file_gold = os.path.join(root_txt,"en.test.gold.v1.1.txt")
    file_data = os.path.join(root_txt,"en.test.data.v1.1.txt")
elif language == "persian":
    file_gold = os.path.join(root_txt,"fa.test.gold.txt")
    file_data = os.path.join(root_txt,"fa.test.data.txt")
elif language == "italian":
    file_gold = os.path.join(root_txt,"it.test.gold.v1.1.txt")
    file_data = os.path.join(root_txt,"it.test.data.v1.1.txt") 
    

In [14]:
data = pd.read_csv(file_data, delimiter='\t', header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,goal,football goal,image.4418.jpg,image.4416.jpg,image.4417.jpg,image.4413.jpg,image.4412.jpg,image.4415.jpg,image.4419.jpg,image.4414.jpg,image.2166.jpg,image.1150.jpg
1,mustard,mustard seed,image.4429.png,image.4422.jpg,image.4423.jpg,image.4424.jpg,image.4421.jpg,image.4427.jpg,image.4426.jpg,image.4420.jpg,image.4425.jpg,image.4428.jpg
2,seat,eating seat,image.4435.jpg,image.4436.jpg,image.1166.jpg,image.4430.jpg,image.4433.jpg,image.4432.jpg,image.4438.jpg,image.4434.jpg,image.4431.jpg,image.4437.jpg
3,navigate,navigate the web,image.4439.jpg,image.4440.jpg,image.4441.jpg,image.4442.jpg,image.4444.jpg,image.4445.jpg,image.1435.jpg,image.4446.png,image.1434.jpg,image.4443.jpg
4,butterball,butterball person,image.4454.jpg,image.4450.jpg,image.4455.jpg,image.4453.jpg,image.4448.jpg,image.1253.jpg,image.4451.jpg,image.4452.jpg,image.4447.jpg,image.4449.jpg


In [72]:
df_gold = pd.read_csv(file_gold, delimiter='\t', header=None)
df_gold.head()

,0
0,image.2166.jpg
1,image.4429.png
2,image.4432.jpg
3,image.1435.jpg
4,image.4455.jpg


## Model Definition

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
device

device(type='cpu')

In [8]:
max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

#### Generate the description for the image

In [41]:
img_list_0 = [item[2:] for item in data.values.tolist()]
img_list_1 = []
for item in img_list_0:
    for item2 in item:
        img_list_1.append(item2)

In [47]:
# get unique values / image names
img_list = set(img_list_1)
img_list = list(img_list)

In [48]:
img_list

['image.5848.jpg',
 'image.6307.jpg',
 'image.5680.jpg',
 'image.4827.jpg',
 'image.4469.jpg',
 'image.6647.jpg',
 'image.7720.jpg',
 'image.5895.jpg',
 'image.4753.jpg',
 'image.7159.jpg',
 'image.5363.jpg',
 'image.4598.jpg',
 'image.7558.jpg',
 'image.5443.jpg',
 'image.4714.jpg',
 'image.6393.jpg',
 'image.5414.jpg',
 'image.7182.jpg',
 'image.3721.jpg',
 'image.5507.jpg',
 'image.4677.jpg',
 'image.6684.JPG',
 'image.2435.jpg',
 'image.6602.jpg',
 'image.4435.jpg',
 'image.4480.jpg',
 'image.7379.jpg',
 'image.5940.jpg',
 'image.4477.jpg',
 'image.6058.jpg',
 'image.5516.jpg',
 'image.6524.jpg',
 'image.6839.jpg',
 'image.5088.jpg',
 'image.6038.jpg',
 'image.8071.jpg',
 'image.7461.jpg',
 'image.435.jpg',
 'image.7165.jpg',
 'image.6100.jpg',
 'image.5229.jpg',
 'image.6380.jpg',
 'image.4657.jpg',
 'image.4635.jpg',
 'image.5962.jpg',
 'image.7380.jpg',
 'image.4644.jpg',
 'image.257.jpg',
 'image.5403.jpg',
 'image.5369.jpg',
 'image.4767.jpg',
 'image.4980.jpg',
 'image.7579.j

In [ ]:
zipFile = zipfile.ZipFile("/content/drive/MyDrive/semeval-2023-task-1-V-WSD-train-v1.zip")
IMG_PATH = "semeval-2023-task-1-V-WSD-train-v1"

In [52]:
def get_images_description(image_path, image_names):
    images = list()
    for img in image_names:
        image = Image.open(os.path.join(image_path, img))

        if image.mode != "RGB":
            image = image.convert("RGB")
        
        images.append(image)
    
    model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
    model.to(device)
    feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
    tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
    
    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = model.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]

    del pixel_values
    del output_ids
    del model
    
    torch.cuda.empty_cache()
    
    return preds

In [44]:
description_path = "Descriptions"
if not os.path.exists(description_path):
    os.makedirs(description_path)

In [53]:
batch_size = 10
start = 0
end = 10
descriptions = list()

for i in range(round(len(img_list)/batch_size)):
    print(f"Working on Data[{start}:{end}]")
    image_names = img_list[start:end]
    desc = get_images_description(folder_images, image_names)
    descriptions.extend(desc)
    start += 10
    end += 10
df = pd.DataFrame({"image": img_list, "description": descriptions})
df.to_csv(os.path.join(description_path,"description.csv"), index=False)

Working on Data[0:10]


C:\Users\Insaf\anaconda3\envs\competition01\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Insaf\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.4.attn.bias', 'decoder.transformer.h.7.crossattention.masked_bias', 'decoder.transformer.h.9.attn.bias', 'decoder.transformer.h.1.crossattention.masked_bias', 'decoder.transformer.h.0.attn.bias', 'decoder.transformer.h.9.attn.masked_bias', 'decoder.transformer.h.10.crossattention.masked_bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.7.crossattention.bias', 'decoder.transformer.h.1.crossattention.bias', 'decoder.transformer.h.10.crossattention.bias', 'decoder.transformer.h.3.crossattention.bias', 'decoder.transformer.h.3.attn.masked_bias', 'decoder.transformer.h.11.crossattention.bias', 'decoder.transformer.h.6.crossattention.masked_bias', 'decoder.transformer.h.9.crossattention.bias', 'decoder.transformer.h.6.attn.masked_bias', 'decoder.transformer.h.7.attn.bias', 'decoder.transformer.h.10.attn.bias', 

C:\Users\Insaf\anaconda3\envs\competition01\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:31: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  FutureWarning,


Working on Data[10:20]


...

Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.4.attn.bias', 'decoder.transformer.h.7.crossattention.masked_bias', 'decoder.transformer.h.9.attn.bias', 'decoder.transformer.h.1.crossattention.masked_bias', 'decoder.transformer.h.0.attn.bias', 'decoder.transformer.h.9.attn.masked_bias', 'decoder.transformer.h.10.crossattention.masked_bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.7.crossattention.bias', 'decoder.transformer.h.1.crossattention.bias', 'decoder.transformer.h.10.crossattention.bias', 'decoder.transformer.h.3.crossattention.bias', 'decoder.transformer.h.3.attn.masked_bias', 'decoder.transformer.h.11.crossattention.bias', 'decoder.transformer.h.6.crossattention.masked_bias', 'decoder.transformer.h.9.crossattention.bias', 'decoder.transformer.h.6.attn.masked_bias', 'decoder.transformer.h.7.attn.bias', 'decoder.transformer.h.10.attn.bias', 

Working on Data[4070:4080]


Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.4.attn.bias', 'decoder.transformer.h.7.crossattention.masked_bias', 'decoder.transformer.h.9.attn.bias', 'decoder.transformer.h.1.crossattention.masked_bias', 'decoder.transformer.h.0.attn.bias', 'decoder.transformer.h.9.attn.masked_bias', 'decoder.transformer.h.10.crossattention.masked_bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.7.crossattention.bias', 'decoder.transformer.h.1.crossattention.bias', 'decoder.transformer.h.10.crossattention.bias', 'decoder.transformer.h.3.crossattention.bias', 'decoder.transformer.h.3.attn.masked_bias', 'decoder.transformer.h.11.crossattention.bias', 'decoder.transformer.h.6.crossattention.masked_bias', 'decoder.transformer.h.9.crossattention.bias', 'decoder.transformer.h.6.attn.masked_bias', 'decoder.transformer.h.7.attn.bias', 'decoder.transformer.h.10.attn.bias', 

Working on Data[4080:4090]


Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.4.attn.bias', 'decoder.transformer.h.7.crossattention.masked_bias', 'decoder.transformer.h.9.attn.bias', 'decoder.transformer.h.1.crossattention.masked_bias', 'decoder.transformer.h.0.attn.bias', 'decoder.transformer.h.9.attn.masked_bias', 'decoder.transformer.h.10.crossattention.masked_bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.7.crossattention.bias', 'decoder.transformer.h.1.crossattention.bias', 'decoder.transformer.h.10.crossattention.bias', 'decoder.transformer.h.3.crossattention.bias', 'decoder.transformer.h.3.attn.masked_bias', 'decoder.transformer.h.11.crossattention.bias', 'decoder.transformer.h.6.crossattention.masked_bias', 'decoder.transformer.h.9.crossattention.bias', 'decoder.transformer.h.6.attn.masked_bias', 'decoder.transformer.h.7.attn.bias', 'decoder.transformer.h.10.attn.bias', 

In [56]:
df.to_csv(os.path.join(description_path,"description.csv"), index=False)

### Calculate Cosine Scores

In [57]:
result_path = "Results"
if not os.path.exists(result_path):
    os.makedirs(result_path)

In [58]:
descriptions = pd.read_csv(description_path+"description.csv")
descriptions.head()

,image,description
0,image.5848.jpg,a number of small birds in a field
1,image.6307.jpg,a small group of flowers in a flower pot
2,image.5680.jpg,a vase filled with flowers on top of a table
3,image.4827.jpg,a statue of a man with a knife in his mouth
4,image.4469.jpg,a small white bird standing on top of a white ...


In [ ]:
data = pd.read_csv(PATH+f"data_split_for_{NAME}.csv")
data.head()

In [59]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,goal,football goal,image.4418.jpg,image.4416.jpg,image.4417.jpg,image.4413.jpg,image.4412.jpg,image.4415.jpg,image.4419.jpg,image.4414.jpg,image.2166.jpg,image.1150.jpg
1,mustard,mustard seed,image.4429.png,image.4422.jpg,image.4423.jpg,image.4424.jpg,image.4421.jpg,image.4427.jpg,image.4426.jpg,image.4420.jpg,image.4425.jpg,image.4428.jpg
2,seat,eating seat,image.4435.jpg,image.4436.jpg,image.1166.jpg,image.4430.jpg,image.4433.jpg,image.4432.jpg,image.4438.jpg,image.4434.jpg,image.4431.jpg,image.4437.jpg
3,navigate,navigate the web,image.4439.jpg,image.4440.jpg,image.4441.jpg,image.4442.jpg,image.4444.jpg,image.4445.jpg,image.1435.jpg,image.4446.png,image.1434.jpg,image.4443.jpg
4,butterball,butterball person,image.4454.jpg,image.4450.jpg,image.4455.jpg,image.4453.jpg,image.4448.jpg,image.1253.jpg,image.4451.jpg,image.4452.jpg,image.4447.jpg,image.4449.jpg


In [75]:
data2 = pd.DataFrame()

In [82]:
data2["Index"] =  range(data.shape[0]) 
data2["keyword"] = data[0]
data2["context"] = data[1]
data2["img_1"] = data[2]
data2["img_2"] = data[3]
data2["img_3"] = data[4]
data2["img_4"] = data[5]
data2["img_5"] = data[6]
data2["img_6"] = data[7]
data2["img_7"] = data[8]
data2["img_8"] = data[9]
data2["img_9"] = data[10]
data2["img_10"] = data[11]
data2["gold_key"] = df_gold

In [83]:
data2.head()

,Index,keyword,context,img_1,img_2,img_3,img_4,img_5,img_6,img_7,img_8,img_9,img_10,gold_key
0,0,goal,football goal,image.4418.jpg,image.4416.jpg,image.4417.jpg,image.4413.jpg,image.4412.jpg,image.4415.jpg,image.4419.jpg,image.4414.jpg,image.2166.jpg,image.1150.jpg,image.2166.jpg
1,1,mustard,mustard seed,image.4429.png,image.4422.jpg,image.4423.jpg,image.4424.jpg,image.4421.jpg,image.4427.jpg,image.4426.jpg,image.4420.jpg,image.4425.jpg,image.4428.jpg,image.4429.png
2,2,seat,eating seat,image.4435.jpg,image.4436.jpg,image.1166.jpg,image.4430.jpg,image.4433.jpg,image.4432.jpg,image.4438.jpg,image.4434.jpg,image.4431.jpg,image.4437.jpg,image.4432.jpg
3,3,navigate,navigate the web,image.4439.jpg,image.4440.jpg,image.4441.jpg,image.4442.jpg,image.4444.jpg,image.4445.jpg,image.1435.jpg,image.4446.png,image.1434.jpg,image.4443.jpg,image.1435.jpg
4,4,butterball,butterball person,image.4454.jpg,image.4450.jpg,image.4455.jpg,image.4453.jpg,image.4448.jpg,image.1253.jpg,image.4451.jpg,image.4452.jpg,image.4447.jpg,image.4449.jpg,image.4455.jpg


In [84]:
result = pd.DataFrame(columns=['img_1', 'img_2', 'img_3', 'img_4', 'img_5', 'img_6', 'img_7', 'img_8', 'img_9', 'img_10', 'label'])

In [85]:
sentence_model =  SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [86]:
for idx, row in data2.iterrows():
    embeddings_1 = sentence_model.encode(row['context'], convert_to_tensor=True)
    scores = list()
    for i in range(1, 11):
        desc = descriptions[descriptions['image'] == row[f'img_{i}']]['description'].values[0]
        embeddings_2 = sentence_model.encode(desc, convert_to_tensor=True)
        cosine_scores = util.pytorch_cos_sim(embeddings_1, embeddings_2).item()
        
        scores.append(cosine_scores)
        result.loc[idx, f'img_{i}'] = cosine_scores

        del embeddings_2
    result.loc[idx, 'label'] = row[f"img_{np.argmax(scores) + 1}"]
    del embeddings_1
result.to_csv(os.path.join(result_path,"similarities_and_prediction.csv"), index=False)

In [91]:
row

Index                  462
keyword            diamond
context      diamond field
img_1       image.8090.jpg
img_2       image.8091.jpg
img_3       image.8092.jpg
img_4       image.8093.jpg
img_5       image.8098.jpg
img_6       image.8097.jpg
img_7       image.8094.jpg
img_8       image.8099.jpg
img_9       image.8095.jpg
img_10      image.8096.jpg
gold_key    image.8095.jpg
Name: 462, dtype: object

In [137]:
final_result_images = []
final_result_distances = []
for idx, row in result.iterrows():
    dist_test = np.array(result.iloc[idx][:-1])
    ranked_dist = -np.sort(-dist_test)
    ranked_dist_indices = np.argsort(-dist_test)
    sorted_df = np.array(data.iloc[idx][ranked_dist_indices+2])
    final_result_images.append(sorted_df)
    final_result_distances.append(ranked_dist)

        

In [138]:
result_tocsv = pd.DataFrame()
result_tocsv["language"]=[]
result_tocsv["data"]=[]
result_tocsv["candidate"]=final_result_images
result_tocsv["relevance"]=final_result_distances
result_tocsv["language"]=language
result_tocsv["gold"]=df_gold.values
result_tocsv["data"]=range(len(final_result_distances))

In [141]:
result_tocsv

,language,data,candidate,relevance,gold
0,english,0,"[image.4415.jpg, image.2166.jpg, image.4418.jp...","[0.421658992767334, 0.3936811685562134, 0.3816...",image.2166.jpg
1,english,1,"[image.4429.png, image.4423.jpg, image.4427.jp...","[0.2836892902851105, 0.27905523777008057, 0.27...",image.4429.png
2,english,2,"[image.4430.jpg, image.4434.jpg, image.4438.jp...","[0.24093304574489594, 0.20179305970668793, 0.1...",image.4432.jpg
3,english,3,"[image.1435.jpg, image.4445.jpg, image.4441.jp...","[0.26168426871299744, 0.11431465297937393, 0.0...",image.1435.jpg
4,english,4,"[image.4450.jpg, image.4455.jpg, image.1253.jp...","[0.34032198786735535, 0.24167302250862122, 0.2...",image.4455.jpg
...,...,...,...,...,...
458,english,458,"[image.8066.jpg, image.1604.jpg, image.8064.jp...","[0.3713851571083069, 0.16832831501960754, 0.15...",image.8063.jpg
459,english,459,"[image.8072.jpg, image.8071.jpg, image.8076.jp...","[0.4677559435367584, 0.3392558693885803, 0.337...",image.8069.jpg
460,english,460,"[image.8081.jpg, image.4995.jpg, image.8080.jp...","[0.473016619682312, 0.08488843590021133, 0.066...",image.8081.jpg
461,english,461,"[image.8089.jpg, image.7279.jpg, image.192.jpg...","[0.21371154487133026, 0.20012266933918, 0.1846...",image.8089.jpg


In [142]:
# save the csv of results
       
result_name =os.path.join(result_path, "test_sorting2.csv" )
result_tocsv.to_csv(result_name)


### Accuracy

In [90]:
accuracy_score(data2['gold_key'], result['label'])

0.3045356371490281